In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
path = 'data_filter_1.csv'

In [3]:
from openai import OpenAI # openai==1.52.2

client = OpenAI(
    api_key="up_EKKzV9chKEmZbAI8OpmWXpww0JdKq",
    base_url="https://api.upstage.ai/v1"
)
def openai_stream(content):
    stream = client.chat.completions.create(
        model="solar-pro2",
        messages=[
            {
                "role": "system",
                "content": content
            }
        ],
        stream=True,
    )
    full_response = ""

    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta.content is not None:
            full_response += delta.content  # 누적 저장

    return full_response



In [4]:

data = pd.read_csv(path)
output = []
count = 1

pbar = tqdm(total=data['output'].isnull().sum())
for content ,valid_output in data[['prompt','output']].values:
    if pd.isna(valid_output):
        try:
            pbar.update(1)
            count+=1
            result = openai_stream(content)
            output.append(result)
        except Exception as e:
            print(e)
            output.append(None)
    else:
        output.append(valid_output)
    if count%200==0:
        data['output'] = output+([None] * (data.shape[0]-len(output)))
        data.to_csv(path, index=False)
pbar.close()

  0%|          | 0/847 [00:00<?, ?it/s]

100%|██████████| 847/847 [2:41:13<00:00, 11.42s/it]  


In [5]:
data['output'] = output + ([None] * (data.shape[0]-len(output)))
data.to_csv(path, index=False)